

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_UR_FAKENEWS.ipynb)






# **Fake News Classifier in Urdu**



## 1. Colab Setup

In [ ]:
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

## 2. Start Spark Session

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## 3. Define Spark NLP pipeline

In [4]:
text_list = ["""فراہم کرے گاڈان اخبار کی رپورٹ کے مطابق یہ جولائی 2019 میں ایک کروڑ 50 لاکھ ڈالر کی منظوری کے بعد سے منصوبے کی تیسری فنانسنگ ہوگییہ بھی پڑھیں ئندہ سال تک پاکستان میں غربت میں اضافے کا امکان ہے عالمی""","""ٹرمپ سونامی کی آمد: ٹرمپ نے آج فلوریڈا میں کیا کیا وہ انہیں صدر بنا دے گا! ٹرمپ سونامی کی آمد: ٹرمپ نے آج فلوریڈا میں کیا کیا وہ انہیں صدر بنا دے گا! فشر 5 منٹ پہلے خبریں تبصرے بند آن ٹرمپ سونامی آنے والی: ٹرمپ نے آج فلوریڈا میں کیا کیا وہ اسے صدر بنا دے گا! ٹرمپ سونامی کی آمد: ٹرمپ نے آج فلوریڈا میں کیا کیا وہ انہیں صدر بنا دے گا! توڑنے! توڑنے! فلوریڈا میں ہلیری کے لیے بری خبر۔ فلوریڈا سے ووٹنگ کے ابتدائی اعداد و شمار ظاہر کر رہے ہیں کہ ریپبلکنز نے 17,000 مزید ووٹ ڈالے ہیں۔""","""ایک امریکی تھنک ٹینک نے خبردار کیا ہے کہ جیسے جیسے چین مصنوعی ذہانت (آرٹیفیشل انٹیلی جنس) کے میدان میں ترقی کر رہا ہے، دنیا کا اقتصادی اور عسکری توازن تبدیل ہو سکتا ہے۔""","""فلپ کے ناول، کاپی کیٹس اور ہر چیز کی طرح لگتا ہے۔ اقوام متحدہ تمام انسانوں کو بائیو میٹرک شناختی کارڈ سے متاثر کرنے کا منصوبہ بنا رہی ہے۔""","""صدر براک اوباما نے کہا کہ سابق وزیر خارجہ ہلیری کلنٹن کا ذاتی ای میل سرور کا استعمال ایک غلطی تھی، لیکن یہ کہ امریکی قومی سلامتی خطرے میں نہیں پڑی تھی۔ ڈیموکریٹک صدارتی پرائمری کو ہلا دینے والے تنازعہ پر اپنے پہلے وسیع ریمارکس میں، مسٹر اوباما نے CBS کے 60 منٹس" پروگرام میں کہا کہ مسز کلنٹن کے ای میل انتظامات کے بارے میں سوالات جائز تھے۔ "امریکی عوام کے اطمینان کے لیے ان کے لیے ان سوالات کا جواب دینا ضروری ہے،" مسٹر اوباما""","""ڈونلڈ ٹرمپ کی کھانے کی عادات ان کی صحت اور ہماری حفاظت کو ڈرامائی طور پر متاثر کر سکتی ہیں"""]

In [6]:
document_assembler = DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

lemma = LemmatizerModel.pretrained("lemma", "ur") \
         .setInputCols(["normalized"]) \
         .setOutputCol("lemma")

embeddings = WordEmbeddingsModel.pretrained("urduvec_140M_300d", "ur") \
      .setInputCols("document", "lemma") \
      .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classifierdl = ClassifierDLModel.pretrained("classifierdl_urduvec_fakenews", "ur") \
  .setInputCols(["sentence_embeddings"]) \
  .setOutputCol("class_")

urdu_fake_pipeline = Pipeline(
    stages=[
        document_assembler, 
        tokenizer, normalizer, 
        lemma, embeddings, 
        embeddingsSentence, 
        classifierdl])

df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = urdu_fake_pipeline.fit(df).transform(df)


lemma download started this may take some time.
Approximate size to download 111 KB
[OK!]
urduvec_140M_300d download started this may take some time.
Approximate size to download 110.2 MB
[OK!]
classifierdl_urduvec_fakenews download started this may take some time.
Approximate size to download 20.5 MB
[OK!]


## Visualize Results

In [7]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|document                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |class|
+-----------------------